In [ ]:
import urllib2, urllib, json

In [ ]:
def get_yr_built(address):
    stuff_data = {
        'f': 'json',
        'searchText': address,
        'contains': 'false',
        'returnGeometry': 'true',
        'layers': '15',
        'searchFields': 'ADDRESSSIMPLE',
        'sr': '102113',
    }

    info = urllib2.urlopen('http://50.17.237.182/arcgis/rest/services/PIM_v3_4/MapServer/find?' + urllib.urlencode(stuff_data))
    loaded = json.load(info)

    geometry = loaded['results'][0]['geometry']
    
    buffer_data = {
        'f': 'json',
        'unit': '9002',
        'unionResults': 'true',
        'geodesic': 'false',
        'geometries': json.dumps({'geometryType': 'esriGeometryPolygon', 'geometries': [geometry]}),
        'inSR': '102113',
        'distances': '1,-0.95'
    }

    info = urllib2.urlopen('http://50.17.237.182/arcgis/rest/services/Utilities/Geometry/GeometryServer/buffer?' + urllib.urlencode(buffer_data))
    buffer_loaded = json.load(info)
    geometry = {'rings': buffer_loaded['geometries'][-1]['rings'], 'spatialReference': geometry['spatialReference']}

    xvals = [x[0] for x in loaded['results'][0]['geometry']['rings'][0]]
    yvals = [y[0] for y in loaded['results'][0]['geometry']['rings'][0]]
    xmin = min(xvals)
    xmax = max(xvals)
    ymin = min(yvals)
    ymax = max(yvals)
    extent = {'xmin': xmin, 'xmax': xmax, 'ymin': ymin, 'ymax': ymax, 'spatialReference': {'latestWkid': 3857, 'wkid': 102110}}
    
    data = {
        'f': 'json',
        'tolerance': '0',
        'mapExtent': json.dumps(extent),
        'imageDisplay': '392,494,96',
        'geometry': json.dumps(geometry),
        'returnGeometry': False,
        'geometryType': 'esriGeometryPolygon',
        'sr': '102113',
        'layers': 'all:0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,61,62,63,64,66,67,68,69,70,71,72,73,74,75,76',
    }
    info = urllib2.urlopen('http://50.17.237.182/arcgis/rest/services/PIM_v3_4/MapServer/identify', urllib.urlencode(data))
    result = json.load(info)
    
    stories, lotarea, bldgsqft, units, yrbuilt = 0,0,0,0,0
    for res in result['results']:
        if 'STORIES' in res['attributes'] and res['attributes']['STORIES'] not in ('', 'Null'):
            n_stories = res['attributes']['STORIES']
            if n_stories[-1] < '0' or n_stories[-1] > '9':
                n_stories = n_stories[:-1]
            if n_stories != '':
                stories = int(n_stories)
        if 'UNITS' in res['attributes'] and res['attributes']['UNITS'] not in (0, 'Null'):
            units = int(res['attributes']['UNITS'])
        if 'BLDGSQFT' in res['attributes'] and res['attributes']['BLDGSQFT'] != 'Null':
            bldgsqft = float(res['attributes']['BLDGSQFT'])
        if 'LOTAREA' in res['attributes'] and res['attributes']['LOTAREA'] != 'Null':
            lotarea = float(res['attributes']['LOTAREA'])
        if 'YRBUILT' in res['attributes'] and res['attributes']['YRBUILT'] != 'Null':
            yrbuilt = int(res['attributes']['YRBUILT'])
    return stories, units, bldgsqft, lotarea, yrbuilt

In [ ]:
import bz2
with bz2.BZ2File('data/citylots.csv.bz2') as f:
    file_content = f.readlines()
len(file_content)

In [ ]:
import random
content = random.sample(file_content, 1000)

In [ ]:
result = []
for index, house in enumerate(content):
    _,_,_,_,beg,end,street,st_type,_ = house.split(',')
    if street == '':
        continue

    addr = '{} {} {}'.format(beg, street, st_type)
    print '{}: {}...'.format(index, addr)

    try:
        built = get_yr_built(addr)
        if built == (0,0,0,0,0):
            continue
    except Exception:
        addr = '{} {} {}'.format(end, street, st_type)
        print '{}: {}...'.format(index, addr)

        try:
            built = get_yr_built(addr)
            if built == (0,0,0,0,0):
                continue
        except Exception:
            continue
        else:
            print ' ==>', built
            result.append((addr, built))
    else:
        print ' ==>', built
        result.append((addr, built))

In [ ]:
import pickle

In [ ]:
pickle.dump(result, open('addresses.pkl', 'w'))